In [ ]:
import glob
import pandas as pd
import cx_Oracle
conn = cx_Oracle.connect('/@db')


In [ ]:
def exec_query(query, conn):
    cur = conn.cursor()
    cur.execute(query)
    conn.commit()
    cur.close()

In [ ]:
create_tbl_query = """
CREATE TABLE world_vacc_customers (
Name VARCHAR(255) PRIMARY KEY,
Cust_I VARCHAR(18) NOT NULL,
Open_Dt DATE NOT NULL,
Consul_Dt DATE,
VAC_ID CHAR(5),
DR_Name CHAR(255),
State CHAR(5),
County CHAR(5),
DOB DATE,
FLAG CHAR(1)

)

"""

In [ ]:
exec_query(create_tbl_query, conn)

In [ ]:
insert_sql='INSERT INTO world_vacc_customers VALUES(:1,:2,:3,:4,:5,:6,:7,:8,:9,:10)'

In [ ]:
def insert_df_tbl(df,conn,insert_qry):
    rows = [tuple(x) for x in df.values]
    cur = conn.cursor()
    cur.executemany(insert_query, rows)
    conn.commit()
    cur.close()

In [ ]:
data_files = glob.glob("./*.csv")
for file in data_files:
    data = pd.read_csv(file, sep='|', index_col=False)
    data.drop(['Unnamed: 0','H'],inplace=True,axis=1)
    insert_df_tbl(data,conn,insert_sql)
    

In [ ]:
create_country_tbl_qry = """

CREATE TABLE Table_{country_name} AS 
SELECT * FROM world_vacc_customers WHERE UPPER(County)={country_name}

"""

In [ ]:
unique_countries = pd.read_sql_query("""
SELECT DISTINCT UPPER(County) FROM world_vacc_customers
""",conn)

In [ ]:
for country in unique_countries:
    create_country_tbl_qry = create_country_tbl_qry.format(country_name=country)
    exec_query(create_country_tbl_qry, conn)

In [ ]:
conn.close()